<a href="https://colab.research.google.com/github/Raghul847/ML-Basic-project-/blob/main/23f2002365_notebook_t32025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

f=pd.read_csv('/content/sample_data/booknow_booking.csv')
f1=pd.read_csv('/content/sample_data/booknow_theaters.csv')
f2= pd.read_csv('/content/sample_data/booknow_visits.csv')
f3= pd.read_csv('/content/sample_data/cinePOS_theaters.csv')
f4=pd.read_csv('/content/sample_data/date_info.csv')
f5= pd.read_csv('/content/sample_data/movie_theater_id_relation.csv')


f0=pd.merge(f,f1,on=['book_theater_id'])
f0=pd.merge(f0,f2,on=['book_theater_id'])
f0=pd.merge(f0,f4,on=['show_date'])
f0=pd.merge(f0,f5,on=['book_theater_id'])
f0=pd.merge(f0,f3,on=['cine_theater_id'])
f0['ID']=f0['book_theater_id']+'_'+f0['show_date']
f0

,book_theater_id,show_datetime,booking_datetime,tickets_booked,theater_type_x,theater_area_x,latitude_x,longitude_x,show_date,audience_count,day_of_week,cine_theater_id,theater_type_y,theater_area_y,latitude_y,longitude_y,ID
0,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,2023-01-01,16,Sunday,cinePOS_00091,Other,Area_141,NaN,NaN,book_00151_2023-01-01
1,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,2023-01-02,188,Monday,cinePOS_00091,Other,Area_141,NaN,NaN,book_00151_2023-01-02
2,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,2023-01-03,138,Tuesday,cinePOS_00091,Other,Area_141,NaN,NaN,book_00151_2023-01-03
3,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,2023-01-04,48,Wednesday,cinePOS_00091,Other,Area_141,NaN,NaN,book_00151_2023-01-04
4,book_00151,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,2023-01-05,10,Thursday,cinePOS_00091,Other,Area_141,NaN,NaN,book_00151_2023-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1926619,book_00122,2024-02-28 22:00:00,2024-02-27 22:00:00,3,Other,Area_011,22.997768,79.913144,2024-02-24,28,Saturday,cinePOS_01048,Other,Area_108,NaN,NaN,book_00122_2024-02-24
1926620,book_00122,2024-02-28 22:00:00,2024-02-27 22:00:00,3,Other,Area_011,22.997768,79.913144,2024-02-25,188,Sunday,cinePOS_01048,Other,Area_108,NaN,NaN,book_00122_2024-02-25
1926621,book_00122,2024-02-28 22:00:00,2024-02-27 22:00:00,3,Other,Area_011,22.997768,79.913144,2024-02-26,68,Monday,cinePOS_01048,Other,Area_108,NaN,NaN,book_00122_2024-02-26
1926622,book_00122,2024-02-28 22:00:00,2024-02-27 22:00:00,3,Other,Area_011,22.997768,79.913144,2024-02-27,18,Tuesday,cinePOS_01048,Other,Area_108,NaN,NaN,book_00122_2024-02-27


In [ ]:
# 1) Feature engineering from datetimes
f0 = f0.copy()
for col in ['show_datetime','booking_datetime']:
    if col in f0.columns:
        f0[col] = pd.to_datetime(f0[col], errors='coerce')

if 'show_datetime' in f0.columns:
    f0['show_hour'] = f0['show_datetime'].dt.hour
    f0['show_dow'] = f0['show_datetime'].dt.dayofweek

if 'booking_datetime' in f0.columns:
    f0['booking_hour'] = f0['booking_datetime'].dt.hour
    f0['booking_dow'] = f0['booking_datetime'].dt.dayofweek


target = 'audience_count'
id_cols = ['ID','book_theater_id','cine_theater_id']
raw_dt_cols = ['show_datetime','booking_datetime']  # remove raw datetimes after feature extraction

drop_cols = [c for c in id_cols + raw_dt_cols if c in f0.columns]
assert target in f0.columns, "Target not found in f0."

X = f0.drop(columns=[target] + drop_cols, errors='ignore')
y = f0[target]

# 3) Split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 4) Preprocess: numeric + categorical
# Exclude datetime columns explicitly
cat_cols = [c for c in X.columns if X[c].dtype == 'object']
num_cols = [c for c in X.columns if X[c].dtype != 'object' and X[c].dtype != 'datetime64[ns]']


preprocess = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')),
                      ('scaler', StandardScaler())]), num_cols),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                      ('ohe', OneHotEncoder(handle_unknown='ignore'))]), cat_cols)
])

model = LinearRegression()

pipe = Pipeline([('prep', preprocess),
                 ('model', model)])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_valid)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_valid, y_pred)
mse = mean_squared_error(y_valid, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_valid, y_pred)
print({'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2})

{'MAE': 21.27243471234064, 'MSE': 860.1265468009194, 'RMSE': np.float64(29.32791412291231), 'R2': 0.35537771619115766}


In [ ]:
# Load the test data
f7 = pd.read_csv('/content/sample_data/sample_submission.csv')

# Preprocess f7 (sample_submission)
# Safely extract book_theater_id and show_date from the 'ID' column
# Use regex to extract the book_theater_id and the date part
id_pattern = r'^(book_\d+)_(\d{4}-\d{2}-\d{2})$'
extracted_data = f7['ID'].str.extract(id_pattern)

# Assign the extracted parts to new columns
f7['book_theater_id'] = extracted_data[0]
f7['show_date'] = pd.to_datetime(extracted_data[1])

# Drop rows where extraction failed (if any)
f7.dropna(subset=['book_theater_id', 'show_date'], inplace=True)


# Merge with relevant dataframes to get features similar to training data
# Merge with f1 (booknow_theaters)
test_merged = pd.merge(f7, f1, on='book_theater_id', how='left')

# Merge with f5 (movie_theater_id_relation) - Corrected from f6
test_merged = pd.merge(test_merged, f5, on='book_theater_id', how='left')

# Merge with f3 (cinePOS_theaters) using cine_theater_id
# Print columns of f3 to check for conflicts
print("Columns of f3:", f3.columns)
# The suffixes are needed as both test_merged (from f1) and f3 have theater_type, theater_area, latitude, and longitude columns
test_merged = pd.merge(test_merged, f3, on='cine_theater_id', how='left', suffixes=('_booknow', '_cinepos'))

# Extract date features from show_date for alignment with training data features
test_merged['show_dayofweek_booknow'] = test_merged['show_date'].dt.dayofweek

# Add placeholder columns for features not in the submission data, to be imputed later
for col in ['tickets_booked', 'show_hour_booknow', 'booking_hour_booknow', 'booking_dayofweek_booknow', 'tickets_sold', 'show_hour_cinepos', 'booking_hour_cinepos', 'show_dayofweek_cinepos', 'booking_dayofweek_cinepos']:
    if col not in test_merged.columns:
        test_merged[col] = np.nan

# Display the head of the merged test data
print("Merged test data head:")
display(test_merged.head())
print("\nMerged test data info:")
display(test_merged.info())

Columns of f3: Index(['cine_theater_id', 'theater_type', 'theater_area', 'latitude',
       'longitude'],
      dtype='object')
Merged test data head:


,ID,audience_count,book_theater_id,show_date,theater_type_booknow,theater_area_booknow,latitude_booknow,longitude_booknow,cine_theater_id,theater_type_cinepos,...,show_dayofweek_booknow,tickets_booked,show_hour_booknow,booking_hour_booknow,booking_dayofweek_booknow,tickets_sold,show_hour_cinepos,booking_hour_cinepos,show_dayofweek_cinepos,booking_dayofweek_cinepos
0,book_00001_2024-03-01,0,book_00001,2024-03-01,Drama,Area_002,23.00441,79.934515,NaN,NaN,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,book_00001_2024-03-02,0,book_00001,2024-03-02,Drama,Area_002,23.00441,79.934515,NaN,NaN,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,book_00001_2024-03-03,0,book_00001,2024-03-03,Drama,Area_002,23.00441,79.934515,NaN,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,book_00001_2024-03-04,0,book_00001,2024-03-04,Drama,Area_002,23.00441,79.934515,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,book_00001_2024-03-06,0,book_00001,2024-03-06,Drama,Area_002,23.00441,79.934515,NaN,NaN,...,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Merged test data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38062 entries, 0 to 38061
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   ID                         38062 non-null  object        
 1   audience_count             38062 non-null  int64         
 2   book_theater_id            38062 non-null  object        
 3   show_date                  38062 non-null  datetime64[ns]
 4   theater_type_booknow       14606 non-null  object        
 5   theater_area_booknow       14606 non-null  object        
 6   latitude_booknow           14606 non-null  float64       
 7   longitude_booknow          14606 non-null  float64       
 8   cine_theater_id            7043 non-null   object        
 9   theater_type_cinepos       3066 non-null   object        
 10  theater_area_cinepos       3066 non-null   object        
 11  latitude_cinepos           528 non-null    

None

In [ ]:
# 1) Identify datetime columns present in training X
dt_cols = [c for c in X.columns if np.issubdtype(X[c].dtype, np.datetime64)]

# 2) Ensure test has the same feature engineering applied; if raw datetimes are not modeled directly, drop them
# Option A: drop raw datetime columns to match the pipeline's expectations
X_test_processed = X_valid.drop(columns=dt_cols, errors='ignore')

# Re-add any missing non-datetime columns with correct dtype from training
for c in (set(X.columns) - set(X_test_processed.columns)):
    X_test_processed[c] = X[c].dtype.type(0) if np.issubdtype(X[c].dtype, np.number) else ""

# 3) Ensure column order matches
X_test_processed = X_test_processed[X.columns.drop(dt_cols, errors='ignore')]

# 4) Predict
y_pred_test = pipe.predict(X_test_processed)


In [ ]:

# 1. Align test data columns with training features (X)
X_test_processed = test_merged[[col for col in X.columns if col in test_merged.columns]].copy()

# 2. Add missing columns from X to X_test_processed and fill with a placeholder (e.g., 0)
missing_cols = set(X.columns) - set(X_test_processed.columns)
for c in missing_cols:
    X_test_processed[c] = 0  # You can use mean/mode from X_train if you want

# 3. Ensure the order of columns is the same as in X
X_test_processed = X_test_processed[X.columns]

# 4. Remove all datetime columns from X_test_processed (to avoid DTypePromotionError)
dt_cols = [c for c in X_test_processed.columns if np.issubdtype(X_test_processed[c].dtype, np.datetime64)]
if dt_cols:
    print(f"Dropping datetime columns from test set: {dt_cols}")
    X_test_processed = X_test_processed.drop(columns=dt_cols)
    # Also drop from X.columns to keep alignment
    X_test_processed = X_test_processed[[col for col in X.columns if col not in dt_cols]]

# 5. Impute missing values and scale numeric features using the preprocessor trained on the training data
print("Columns expected by the preprocessor (from X.columns):", X.columns.tolist())
print("Columns in X_test_processed before transformation:", X_test_processed.columns.tolist())

# 6. Make predictions on the preprocessed and aligned test


Dropping datetime columns from test set: ['show_date']
Columns expected by the preprocessor (from X.columns): ['tickets_booked', 'theater_type_x', 'theater_area_x', 'latitude_x', 'longitude_x', 'show_date', 'day_of_week', 'theater_type_y', 'theater_area_y', 'latitude_y', 'longitude_y', 'show_hour', 'show_dow', 'booking_hour', 'booking_dow']
Columns in X_test_processed before transformation: ['tickets_booked', 'theater_type_x', 'theater_area_x', 'latitude_x', 'longitude_x', 'day_of_week', 'theater_type_y', 'theater_area_y', 'latitude_y', 'longitude_y', 'show_hour', 'show_dow', 'booking_hour', 'booking_dow']


In [ ]:
# Define target variable and features
target = 'audience_count'
features = [col for col in f0.columns if col not in ['audience_count', 'book_theater_id', 'show_datetime_booknow', 'booking_datetime_booknow', 'show_date', 'booking_date_booknow', 'cine_theater_id', 'show_datetime_cinepos', 'booking_datetime_cinepos', 'booking_date_cinepos', 'theater_type_booknow', 'theater_area_booknow', 'theater_type_cinepos', 'theater_area_cinepos']]

# Select features and target
X = f0[features]
y = f0[target]

# Drop rows with missing target values
data = pd.concat([X, y], axis=1).dropna(subset=[target])
X = data[features]
y = data[target]


print("Features (X) head:")
display(X.head())
print("\nTarget (y) head:")
display(y.head())

Features (X) head:


,show_datetime,booking_datetime,tickets_booked,theater_type_x,theater_area_x,latitude_x,longitude_x,day_of_week,theater_type_y,theater_area_y,latitude_y,longitude_y,ID,show_hour,show_dow,booking_hour,booking_dow
0,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,Sunday,Other,Area_141,NaN,NaN,book_00151_2023-01-01,20,6,1,6
1,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,Monday,Other,Area_141,NaN,NaN,book_00151_2023-01-02,20,6,1,6
2,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,Tuesday,Other,Area_141,NaN,NaN,book_00151_2023-01-03,20,6,1,6
3,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,Wednesday,Other,Area_141,NaN,NaN,book_00151_2023-01-04,20,6,1,6
4,2023-01-01 20:00:00,2023-01-01 01:00:00,5,Comedy,Area_003,22.176869,76.191001,Thursday,Other,Area_141,NaN,NaN,book_00151_2023-01-05,20,6,1,6



Target (y) head:


,audience_count
0,16
1,188
2,138
3,48
4,10


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
from google.colab import files

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values in numeric columns only
num_cols = X_train.select_dtypes(include=[np.number]).columns
imputer = SimpleImputer(strategy='mean')
X_train[num_cols] = imputer.fit_transform(X_train[num_cols])
X_test[num_cols] = imputer.transform(X_test[num_cols])

# Drop non-numeric columns (if you don't want to encode them)
X_train_numeric = X_train[num_cols]
X_test_numeric = X_test[num_cols]

# Train the model (no need to filter y_train)
model = LinearRegression()
model.fit(X_train_numeric, y_train)

# Prepare test data for prediction
# Align test data columns with training features
missing_cols = set(X_train_numeric.columns) - set(test_merged.columns)
for c in missing_cols:
    test_merged[c] = X_train_numeric[c].mean()
X_test_processed = test_merged[X_train_numeric.columns]
X_test_processed.fillna(X_train_numeric.mean(), inplace=True)

# Make predictions
y_pred_test = model.predict(X_test_processed)

# Store predictions in the test DataFrame
test_merged['audience_count'] = np.round(y_pred_test).astype(int)
test_merged['audience_count'] = test_merged['audience_count'].clip(lower=0)

# Prepare the submission file
submission_df = test_merged[['ID', 'audience_count']]
submission_df.to_csv('submission.csv', index=False)
files.download('submission.csv')


/tmp/ipython-input-1077771516.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_processed.fillna(X_train_numeric.mean(), inplace=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>